# Análisis de RFM para un conjunto de datos de ventas

## Importación de librerías y carga de datos

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


In [0]:

import numpy as np 
import pandas as pd 
import time, warnings
import datetime as dt

#visualizaciones
import matplotlib.pyplot as plt
from pandas.plotting import scatter_matrix
%matplotlib inline
import seaborn as sns

warnings.filterwarnings("ignore")

In [0]:
#cargar el dataset
retail_df = pd.read_csv('/content/gdrive/My Drive/Datos/RFM_data.csv',encoding="ISO-8859-1",dtype={'CustomerID': str,'InvoiceID': str})
retail_df.head()

## Filtros y limpieza de datos

In [0]:
#Filtramos UK porque el comportamiento puede ser diferente por país
retail_uk = retail_df[retail_df['Country']=='United Kingdom']

retail_uk.shape

(495478, 8)

In [0]:
#eliminamos pedidos cancelados

retail_uk = retail_uk[retail_uk['Quantity']>0]
retail_uk.shape


(486286, 8)

In [0]:
#eliminamos las filas donde el customerID es NA
retail_uk.dropna(subset=['CustomerID'],how='all',inplace=True)
retail_uk.shape

(354345, 8)

In [0]:
#Restringimos a solo un año
retail_uk = retail_uk[retail_uk['InvoiceDate']>= "2010-12-09"]
retail_uk.shape

(176137, 8)

In [0]:
print("Resumen..")
#Explorar los valores únicos de cada atributo
print("Número de transacciones: ", retail_uk['InvoiceNo'].nunique())
print("Número de productos comprados: ",retail_uk['StockCode'].nunique())
print("Número de clientes:", retail_uk['CustomerID'].nunique() )
print("Porcentaje de clientes NA: ", round(retail_uk['CustomerID'].isnull().sum() * 100 / len(retail_df),2),"%" )

Resumen..
Número de transacciones:  8789
Número de productos comprados:  3294
Número de clientes: 2864
Porcentaje de clientes NA:  0.0 %


## Análisis RFM

In [0]:
#última fecha disponible en nuestro conjunto de datos
retail_uk['InvoiceDate'].max()

'9/9/2011 9:52'

In [0]:
now = dt.date(2011,12,9)
print(now)

2011-12-09


In [0]:
#creamos una nueva columna llamada fecha que contiene úincamente la fecha de la factura
retail_uk['date'] = pd.DatetimeIndex(retail_uk['InvoiceDate']).date

In [0]:
retail_uk.head()

In [0]:
#Agrupamos por cliente y chequeamos la fecha de su última compra
recency_df = retail_uk.groupby(by='CustomerID', as_index=False)['date'].max()
recency_df.columns = ['CustomerID','LastPurshaceDate']
recency_df.head()

,CustomerID,LastPurshaceDate
0,12747,2011-08-22
1,12748,2011-09-30
2,12749,2011-08-01
3,12820,2011-09-26
4,12821,2011-05-09


## Cálculo de Recency

In [0]:
#Cálculo de recency
recency_df['Recency'] = recency_df['LastPurshaceDate'].apply(lambda x: (now - x).days)

In [0]:
recency_df.head()

In [0]:
#eliminamos LastPurchaseDate 
recency_df.drop('LastPurshaceDate',axis=1,inplace=True)

## Frecuencia

In [0]:
# eliminamos duplicados 
retail_uk_copy = retail_uk
retail_uk_copy.drop_duplicates(subset=['InvoiceNo', 'CustomerID'], keep="first", inplace=True)
#calculamos la frecuencia de las compras
frequency_df = retail_uk_copy.groupby(by=['CustomerID'], as_index=False)['InvoiceNo'].count()
frequency_df.columns = ['CustomerID','Frequency']
frequency_df.head()

## Monetary

In [0]:
#Creamos una columna de coste total
retail_uk['TotalCost'] = retail_uk['Quantity'] * retail_uk['UnitPrice']

In [0]:
monetary_df = retail_uk.groupby(by='CustomerID',as_index=False).agg({'TotalCost': 'sum'})
monetary_df.columns = ['CustomerID','Monetary']
monetary_df.head()

## Crear tabla RFM

In [0]:
#hacemos un merge de los dataframes de recency y frequency
temp_df = recency_df.merge(frequency_df,on='CustomerID')
temp_df.head()

In [0]:
#hacemos el merge con el dataframe de monetary para obtener una tabla con las tres columnas
rfm_df = temp_df.merge(monetary_df,on='CustomerID')
#usamos CustomerID como índice
rfm_df.set_index('CustomerID',inplace=True)

rfm_df.head()

In [0]:
retail_uk[retail_uk['CustomerID']=='12820']

In [0]:
(now - dt.date(2011,9,26)).days == 74

In [0]:
quantiles = rfm_df.quantile(q=[0.25,0.5,0.75])
quantiles

In [0]:
quantiles.to_dict()

{'Frequency': {0.25: 1.0, 0.5: 2.0, 0.75: 3.0},
 'Monetary': {0.25: 16.35, 0.5: 35.400000000000006, 0.75: 92.42000000000002},
 'Recency': {0.25: 85.0, 0.5: 119.0, 0.75: 183.0}}

## Creación segmentos RFM

In [0]:
# Argumentos (x = value, p = recency, monetary_value, frequency, d = quartiles dict)
def RScore(x,p,d):
    if x <= d[p][0.25]:
        return 4
    elif x <= d[p][0.50]:
        return 3
    elif x <= d[p][0.75]: 
        return 2
    else:
        return 1
# Argumentos (x = value, p = recency, monetary_value, frequency, k = quartiles dict)
def FMScore(x,p,d):
    if x <= d[p][0.25]:
        return 1
    elif x <= d[p][0.50]:
        return 2
    elif x <= d[p][0.75]: 
        return 3
    else:
        return 4

In [0]:
#creamos una tabla de segmentación por rfm
rfm_segmentation = rfm_df
rfm_segmentation['R_Quartile'] = rfm_segmentation['Recency'].apply(RScore, args=('Recency',quantiles,))
rfm_segmentation['F_Quartile'] = rfm_segmentation['Frequency'].apply(FMScore, args=('Frequency',quantiles,))
rfm_segmentation['M_Quartile'] = rfm_segmentation['Monetary'].apply(FMScore, args=('Monetary',quantiles,))

In [0]:
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile
CustomerID,,,,,,
12747,109,5,191.85,3,4,4
12748,70,96,1054.43,4,4,4
12749,130,3,67.00,2,3,3
12820,74,1,15.00,4,1,1
12821,214,1,19.92,1,1,2


In [0]:
rfm_segmentation['RFMScore'] = rfm_segmentation.R_Quartile.map(str) \
                            + rfm_segmentation.F_Quartile.map(str) \
                            + rfm_segmentation.M_Quartile.map(str)
rfm_segmentation.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
12747,109,5,191.85,3,4,4,344
12748,70,96,1054.43,4,4,4,444
12749,130,3,67.00,2,3,3,233
12820,74,1,15.00,4,1,1,411
12821,214,1,19.92,1,1,2,112


In [0]:
rfm_segmentation[rfm_segmentation['RFMScore']=='444'].sort_values('Monetary', ascending=False).head(10)

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,
18102,72,34,26632.62,4,4,4,444
17949,70,32,22504.73,4,4,4,444
17450,70,28,18009.06,4,4,4,444
16029,80,39,15119.49,4,4,4,444
16013,70,24,10402.34,4,4,4,444
12901,81,20,5915.66,4,4,4,444
13798,72,34,4648.80,4,4,4,444
17857,72,12,4644.68,4,4,4,444
13694,71,32,4472.68,4,4,4,444


In [0]:
print("Mejores clientes: ",len(rfm_segmentation[rfm_segmentation['RFMScore']=='444']))
print('Clientes más fieles: ',len(rfm_segmentation[rfm_segmentation['F_Quartile']==4]))
print("Los que más gastan: ",len(rfm_segmentation[rfm_segmentation['M_Quartile']==4]))
print('A los que casi hemos perdido: ', len(rfm_segmentation[rfm_segmentation['RFMScore']=='244']))
print('Clientes perdidos: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='144']))
print('Clientes "baratos" perdidos: ',len(rfm_segmentation[rfm_segmentation['RFMScore']=='111']))

Mejores clientes:  218
Clientes más fieles:  687
Los que más gastan:  716
A los que casi hemos perdido:  52
Clientes perdidos:  5
Clientes "baratos" perdidos:  278


# Jugando a predecir

In [0]:
#hacemos un merge de los datos originales con la clasificación
final_df = retail_uk.merge(rfm_segmentation,on='CustomerID')
final_df.set_index('CustomerID',inplace=True)
final_df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,Country,date,TotalCost,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFMScore
CustomerID,,,,,,,,,,,,,,,,
14620,545220,21955,DOORMAT UNION JACK GUNS AND ROSES,2,3/1/2011 8:30,7.95,United Kingdom,2011-03-01,15.90,205,2,18.18,1,2,2,122
14620,553660,17091A,LAVENDER INCENSE IN TIN,6,5/18/2011 11:58,0.38,United Kingdom,2011-05-18,2.28,205,2,18.18,1,2,2,122
14740,545221,22021,BLUE FELT EASTER EGG BASKET,6,3/1/2011 8:35,1.65,United Kingdom,2011-03-01,9.90,197,3,42.24,1,3,3,133
14740,551349,20971,PINK BLUE FELT CRAFT TRINKET BOX,12,4/28/2011 10:03,1.25,United Kingdom,2011-04-28,15.00,197,3,42.24,1,3,3,133
14740,554842,23109,LED TEA LIGHTS,6,5/26/2011 18:17,2.89,United Kingdom,2011-05-26,17.34,197,3,42.24,1,3,3,133


In [0]:
y = final_df['RFMScore']
drop_col = ['InvoiceNo','Description','InvoiceDate','Country','date','Recency','Frequency','Monetary','R_Quartile','F_Quartile','M_Quartile','RFMScore']
X = final_df.drop(drop_col,axis=1)

## Preparamos los datos y dividimos conjunto de entrenamiento y test

In [0]:
#Encoding de funciones categóricas
categorical =  ['StockCode']

dummies = pd.get_dummies(X[categorical], prefix = categorical, prefix_sep = '_')
X = pd.concat([X, dummies], axis = 1)
# drop original categorical features
X.drop(categorical, axis = 1, inplace = True)


In [0]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [0]:
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier().fit(X_train, y_train)
print('Accuracy of Decision Tree classifier on training set: {:.2f}'
     .format(clf.score(X_train, y_train)))
print('Accuracy of Decision Tree classifier on test set: {:.2f}'
     .format(clf.score(X_test, y_test)))

Accuracy of Decision Tree classifier on training set: 0.70
Accuracy of Decision Tree classifier on test set: 0.20


In [0]:
from sklearn.metrics import confusion_matrix
prediccion = clf.predict(X_test)
confusion_matrix(y_test, prediccion)

array([[  7,   0,   0, ...,   2,   6,  30],
       [  0,   4,   0, ...,   0,   0,  15],
       [  0,   0,   1, ...,   0,   0,   5],
       ...,
       [  3,   1,   0, ...,   1,   1,  10],
       [  3,   2,   0, ...,   0,   5,  32],
       [ 28,  17,   4, ...,   4,   7, 319]])

In [0]:
from sklearn.neighbors import KNeighborsRegressor

#X_train, X_test, y_train, y_test = train_test_split(X_train, y_train, random_state = 0)

knnreg = KNeighborsRegressor(n_neighbors = 5).fit(X_train, y_train)

In [0]:
#Mean Squared Error (MSE)

y_train_pred_knn = knnreg.predict(X_train)
y_test_pred_knn = knnreg.predict(X_test)
print('MSE train: %.3f, test: %.3f' % (mean_squared_error(y_train, y_train_pred_knn),mean_squared_error(y_test, y_test_pred_knn)))

In [0]:
print('R-squared test score: {:.3f}'
     .format(knnreg.score(X_test, y_test)))

## Regresión Lineal

In [0]:
from sklearn import linear_model
regressor = linear_model.LinearRegression()
lr = regressor.fit(X_train, y_train)

In [0]:
#Mean Squared Error (MSE)
from sklearn.metrics import mean_squared_error
y_train_pred_lr = lr.predict(X_train)
y_test_pred_lr = lr.predict(X_test)
print('MSE train: %.3f, test: %.3f' % (mean_squared_error(y_train, y_train_pred_lr),mean_squared_error(y_test, y_test_pred_lr)))

# Si MSE es mayor en test que en train, clara muestra de overfitting

In [0]:
print('R-squared train score: {:.3f}'
     .format(lr.score(X_train, y_train)))
print('R-squared test score: {:.3f}'
     .format(lr.score(X_test, y_test)))
#R-squared conocido también como Coeficiente de determinación. El coeficiente determina la calidad del modelo para replicar los resultados, 
#y la proporción de variación de los resultados que puede explicarse por el modelo
#Es el porcentaje de variacion de la variable dependiente que explica el modelo lineal
